# LBYL versus EAFP

---
In some other languages,
one can not recover from an error,
or it is difficult to recover from an error,
so one tests input before doing something that could provoke the error.
This technique is called **Look Before You Leap** (LBYL)

For example, in a C program, one must avoid dividing by zero.

Below is a C program that divides by numbers read from a file. When it gets the zero, it crashes.

In [1]:
numbers = '''
2
1
0
-1
-2
'''
with open('numbers', 'w') as f:
    f.write(numbers)

In [2]:
c_program_source_code = r'''
#include <stdlib.h>
#include <stdio.h>

int foo(int x)
{
    return 10 / x;
}

int main(int argc, char *argv[])
{
    int x;
    int y;
    
    while (scanf("%d", &x) == 1) {
        y = foo(x);
        printf("foo(%d) --> %d\n", x, y);
    }        
}
'''

with open('foo.c', 'w') as f:
    f.write(c_program_source_code)

!make foo

cc     foo.c   -o foo


In [3]:
!./foo <numbers

foo(2) --> 5
foo(1) --> 10
Floating point exception (core dumped)


Python is similarly vulnerable.

In [4]:
def foo(x):
    return 10 // x

def main(lines):
    for line in lines:
        x = int(line)
        y = foo(x)
        print("foo(%d) --> %d" % (x, y))
        
main(open('numbers').read().split())

foo(2) --> 5
foo(1) --> 10


ZeroDivisionError: integer division or modulo by zero

So one checks before dividing as shown below.

In [5]:
c_program_source_code = r'''
#include <stdlib.h>
#include <stdio.h>

int foo(int x)
{
    if (x == 0)
        return 0;
    else
        return 10 / x;
}

int main(int argc, char *argv[])
{
    int x;
    int y;
    
    while (scanf("%d", &x) == 1) {
        y = foo(x);
        printf("foo(%d) --> %d\n", x, y);
    }        
}
'''

with open('foo.c', 'w') as f:
    f.write(c_program_source_code)

!make foo

cc     foo.c   -o foo


In [6]:
!./foo <numbers

foo(2) --> 5
foo(1) --> 10
foo(0) --> 0
foo(-1) --> -10
foo(-2) --> -5


One can use the same LBYL technique in Python.

In [7]:
def foo(x):
    if x == 0:
        return 0
    else:
        return 10 // x

def main(lines):
    for line in lines:
        x = int(line)
        y = foo(x)
        print("foo(%d) --> %d" % (x, y))
        
main(open('numbers').read().split())

foo(2) --> 5
foo(1) --> 10
foo(0) --> 0
foo(-1) --> -10
foo(-2) --> -5


Another technique is to just try stuff,
and if it blows up, do something else.

This technique is called **Easier to Ask Forgiveness than Permission** (EAFP).

Python makes it very easy to do something else
when something blows up.

In [8]:
def foo(x):
    try:
        y = 10 // x
    except ZeroDivisionError:
        y = 0
    return y

def main(lines):
    for line in lines:
        x = int(line)
        y = foo(x)
        print("foo(%d) --> %d" % (x, y))
        
main(numbers.split())

foo(2) --> 5
foo(1) --> 10
foo(0) --> 0
foo(-1) --> -10
foo(-2) --> -5


For that simple example,
EAFP does not have much if any benefit over LBYL.
For that simple example,
there is not much benefit in the size or readability of the code.
However, for more complicated problems,
EAFP lets one write much simpler and readable code.

We will use the example of determining if a string is a valid float for Python.

See
[2.4.6. Floating point literals](https://docs.python.org/3/reference/lexical_analysis.html#floating-point-literals)
for what constitutes a valid float.

    floatnumber   ::=  pointfloat | exponentfloat
    pointfloat    ::=  [digitpart] fraction | digitpart "."
    exponentfloat ::=  (digitpart | pointfloat) exponent
    digitpart     ::=  digit (["_"] digit)*
    fraction      ::=  "." digitpart
    exponent      ::=  ("e" | "E") ["+" | "-"] digitpart

Some code for that follows.

In [9]:
import re

def is_float(s, debug=False):
    digit = f'([0-9])'
    digitpart = f'({digit}(_?{digit})*)'                           # digit (["_"] digit)*
    fraction = f'([.]{digitpart})'                                 # "." digitpart
    pointfloat = f'(({digitpart}?{fraction}) | ({digitpart}[.]))'  # [digitpart] fraction | digitpart "."
    exponent = f'([eE][-+]?{digitpart})'                           # ("e" | "E") ["+" | "-"] digitpart
    exponentfloat = f'(({digitpart} | {pointfloat}) {exponent})'   # (digitpart | pointfloat) exponent
    floatnumber = f'^({pointfloat} | {exponentfloat})$'            # pointfloat | exponentfloat
    floatnumber = f'^[-+]?({pointfloat} | {exponentfloat} | {digitpart})$'  # allow signs and ints
    
    if debug:
        res = (
            digit,
            digitpart,
            fraction,
            pointfloat,
            exponent,
            exponentfloat,
            floatnumber,
        )
        for s in res:
            print(repr(s))
            # print(str(s))
            
    float_pattern = re.compile(floatnumber, re.VERBOSE)

    return re.match(float_pattern, s)

In [10]:
floats = '''
    2
    0
    -1
    +17.
    .
    -.17
    17e-3
    -19.e-3
    hello
'''.split()

In [11]:
if True:
    _ = is_float('', debug=True)

    for s in floats:
        print(f'{s!r} -> {bool(is_float(s))}')

'([0-9])'
'(([0-9])(_?([0-9]))*)'
'([.](([0-9])(_?([0-9]))*))'
'(((([0-9])(_?([0-9]))*)?([.](([0-9])(_?([0-9]))*))) | ((([0-9])(_?([0-9]))*)[.]))'
'([eE][-+]?(([0-9])(_?([0-9]))*))'
'(((([0-9])(_?([0-9]))*) | (((([0-9])(_?([0-9]))*)?([.](([0-9])(_?([0-9]))*))) | ((([0-9])(_?([0-9]))*)[.]))) ([eE][-+]?(([0-9])(_?([0-9]))*)))'
'^[-+]?((((([0-9])(_?([0-9]))*)?([.](([0-9])(_?([0-9]))*))) | ((([0-9])(_?([0-9]))*)[.])) | (((([0-9])(_?([0-9]))*) | (((([0-9])(_?([0-9]))*)?([.](([0-9])(_?([0-9]))*))) | ((([0-9])(_?([0-9]))*)[.]))) ([eE][-+]?(([0-9])(_?([0-9]))*))) | (([0-9])(_?([0-9]))*))$'
'2' -> True
'0' -> True
'-1' -> True
'+17.' -> True
'.' -> False
'-.17' -> True
'17e-3' -> True
'-19.e-3' -> True
'hello' -> False


In [12]:
import re

def is_float(s, debug=False):
    digit = f'([0-9])'
    digitpart = f'({digit}(_?{digit})*)'                           # digit (["_"] digit)*
    fraction = f'([.]{digitpart})'                                 # "." digitpart
    pointfloat = f'(({digitpart}?{fraction}) | ({digitpart}[.]))'  # [digitpart] fraction | digitpart "."
    exponent = f'([eE][-+]?{digitpart})'                           # ("e" | "E") ["+" | "-"] digitpart
    exponentfloat = f'(({digitpart} | {pointfloat}) {exponent})'   # (digitpart | pointfloat) exponent
    floatnumber = f'^({pointfloat} | {exponentfloat})$'            # pointfloat | exponentfloat
    floatnumber = f'^[-+]?({pointfloat} | {exponentfloat} | {digitpart})$'  # allow signs and ints
    
    float_pattern = re.compile(floatnumber, re.VERBOSE)

    return re.match(float_pattern, s)
    
def safe_float(s, default=0.):
    if is_float(s):
        return float(s)
    else:
        return default

def main(lines):
    total = sum(safe_float(line) for line in lines)
    print(f'total is {total}')
        
main(floats)

total is 17.828


Now we try EAFP technique below.

In [13]:
def safe_float(s, default=0.):
    try:
        x = float(s)
    except ValueError:
        x = default
    return x

def main(lines):
    total = sum(safe_float(line) for line in lines)
    print(f'total is {total}')
        
main(floats)

total is 17.828


The EAFP code is much much simpler.

The LBYL version was very complicated.
If there was a bug in the LBYL version, how would you find it?
If you fixed it, how much confidence would you have that your fix is correct?
How hard would it be to have test cases that covered all the edge cases?

# EAFP makes code easier to read, simpler, and more reliable.

# This is what makes try/except one of Python's superpowers!!!

---

should:

- always specify at least one exception
- put as little as possible in the try clause

Because the 20170227-except-*.py programs can lock up Jupyter notebook,
run them outside of the notebook.

In [14]:
%%script bash
for f in 20170227-except-*.py; do
    python -c "print('#'*79)"
    ls -l "$f"
    echo
    cat "$f"
    echo
done

###############################################################################
-rwxrwxr-x 1 cohpy cohpy 879 Feb 25 19:09 20170227-except-0-bare.py

#!/usr/bin/env python3

'''This program has two try/except sins:
1. too much stuff in try: clause
2. bare except
See https://www.python.org/dev/peps/pep-0008/#programming-recommendations.

Because of the bare except, one can not get out of the program by typing ^C.

For iter(partial(input, prompt), sentinel) idiom,
see https://mail.python.org/pipermail/centraloh/2016-July/002895.html
and http://nbviewer.jupyter.org/github/james-prior/cohpy/blob/master/20160708-dojo-user-input-loop-with-iter-partial-input-prompt-sentinel.ipynb
.
'''

from functools import partial

prompt = 'Please enter an integer: '
while True:
    try:
        for i in map(int, iter(partial(input, prompt), None)):
            print(f'Got {i}. That is a valid integer.')
    except: # This catches all exceptions, including KeyboardInterrupt.
        print('ERROR: That was n